<a href="https://colab.research.google.com/github/YBBBB-C/Grass/blob/main/SpresenseSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import os

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)

    # Check if the image file exists
    if not os.path.exists(image_path):
        raise ValueError(f"Error: Image file not found at path: {image_path}")

    image = cv2.imread(image_path)

    # Check if the image was read successfully
    if image is None:
        raise ValueError(f"Error: Unable to read image from path: {image_path}")

    # Image preprocessing steps
    # For example, we can perform size standardization
    image = cv2.resize(image, (224, 224))

    # Feature extraction steps
    # Assume the color range for bare land in the RGB image is (0, 0, 0) to (100, 100, 100)
    lower_bound = np.array([0, 0, 0], dtype=np.uint8)
    upper_bound = np.array([100, 100, 100], dtype=np.uint8)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Get the mask for bare land regions based on the color range
    bare_land_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Calculate the number of pixels in the bare land region and the total number of pixels to obtain the bare land ratio
    total_pixels = image.shape[0] * image.shape[1]
    bare_pixels = cv2.countNonZero(bare_land_mask)
    bare_ratio = bare_pixels / total_pixels

    # Assume the color range for clover in the HSV image is (40, 40, 40) to (80, 255, 255)
    lower_bound_clover = np.array([40, 40, 40], dtype=np.uint8)
    upper_bound_clover = np.array([80, 255, 255], dtype=np.uint8)

    # Get the mask for clover regions based on the color range
    clover_mask = cv2.inRange(hsv_image, lower_bound_clover, upper_bound_clover)

    # Calculate the number of pixels in the clover region and the total number of pixels to obtain the clover ratio
    clover_pixels = cv2.countNonZero(clover_mask)
    clover_ratio = clover_pixels / total_pixels

    return bare_ratio, clover_ratio


In [ ]:
def generate_labels(bare_ratio, clover_ratio, threshold=0.1):
    if bare_ratio >= threshold and clover_ratio < 0.5:
        return 1  # Suitable for grazing
    else:
        return 0  # Not suitable for grazing


In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


# Image preprocessing and feature extraction function
def preprocess_image(image_path):
    image = cv2.imread(image_path)

    # Check if the image was read successfully
    if image is None:
        raise ValueError(f"Error: Unable to read image from path: {image_path}")

    # Image preprocessing steps
    # For example, resizing the image to (224, 224)
    image = cv2.resize(image, (224, 224))

    # Feature extraction steps
    # Assuming that the color range of bare land in the RGB image is from (0, 0, 0) to (100, 100, 100)
    lower_bound = np.array([0, 0, 0], dtype=np.uint8)
    upper_bound = np.array([100, 100, 100], dtype=np.uint8)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for the bare land region based on the color range
    bare_land_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Calculate the number of pixels in the bare land region and the total number of pixels to get the bare land ratio
    total_pixels = image.shape[0] * image.shape[1]
    bare_pixels = cv2.countNonZero(bare_land_mask)
    bare_ratio = bare_pixels / total_pixels

    # Assuming that the color range of clover in the HSV image is from (40, 40, 40) to (80, 255, 255)
    lower_bound_clover = np.array([40, 40, 40], dtype=np.uint8)
    upper_bound_clover = np.array([80, 255, 255], dtype=np.uint8)

    # Create a mask for the clover region based on the color range
    clover_mask = cv2.inRange(hsv_image, lower_bound_clover, upper_bound_clover)

    # Calculate the number of pixels in the clover region and the total number of pixels to get the clover ratio
    clover_pixels = cv2.countNonZero(clover_mask)
    clover_ratio = clover_pixels / total_pixels

    return bare_ratio, clover_ratio


def generate_labels(bare_ratio, clover_ratio, threshold=0.1):

    print("Bare Ratio:", bare_ratio)
    print("Clover Ratio:", clover_ratio)
    if bare_ratio >= threshold and clover_ratio < 0.5:
        return 1  # Suitable for grazing
    else:
        return 0  # Not suitable for grazing


In [ ]:
# Calculate accuracy
def calculate_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Main function
def main():
    # Prepare image data and labels
    #这里自己改一下路径
    train_image_folder = "/content/drive/MyDrive/grass"  # Path to the folder containing training images
    test_image_folder = "/content/drive/MyDrive/grass"  # Path to the folder containing test images
    train_image_paths = [os.path.join(train_image_folder, image_file) for image_file in os.listdir(train_image_folder)]
    test_image_paths = [os.path.join(test_image_folder, image_file) for image_file in os.listdir(test_image_folder)]
    train_bare_ratios = []
    train_clover_ratios = []
    train_labels = []
    test_bare_ratios = []
    test_clover_ratios = []

    # Process training images
    for image_path in train_image_paths:
        bare_ratio, clover_ratio = preprocess_image(image_path)
        label = generate_labels(bare_ratio, clover_ratio)
        train_bare_ratios.append(bare_ratio)
        train_clover_ratios.append(clover_ratio)
        train_labels.append(label)

    # Process test images
    for image_path in test_image_paths:
        bare_ratio, clover_ratio = preprocess_image(image_path)
        test_bare_ratios.append(bare_ratio)
        test_clover_ratios.append(clover_ratio)

    # Generate CSV files for training and test sets
    train_data = {"Bare_Ratio": train_bare_ratios, "Clover_Ratio": train_clover_ratios, "Label": train_labels}
    test_data = {"Bare_Ratio": test_bare_ratios, "Clover_Ratio": test_clover_ratios, "Label": None}  # Placeholder for test labels
    train_df = pd.DataFrame(train_data)
    test_df = pd.DataFrame(test_data)
    train_df.to_csv("/content/drive/MyDrive/train_data.csv", index=False)
    test_df.to_csv("/content/drive/MyDrive/test_data.csv", index=False)

    # Train the model
    X_train = train_df[["Bare_Ratio", "Clover_Ratio"]]
    y_train = train_df["Label"]
    classifier = SVC()  # Use Support Vector Machine as the classifier
    classifier.fit(X_train, y_train)

    # Use the trained model for prediction
    X_test = test_df[["Bare_Ratio", "Clover_Ratio"]]
    y_pred_test = classifier.predict(X_test)

    # Evaluate the model's performance on the test set and print accuracy
    test_df["Label"] = y_pred_test  # Assign predicted labels to test_df
    test_accuracy = calculate_accuracy(test_df["Label"], y_pred_test)
    train_accuracy = calculate_accuracy(y_train, classifier.predict(X_train))

    print(f"Training accuracy: {train_accuracy}")
    print(f"Test accuracy: {test_accuracy}")

if __name__ == "__main__":
    main()
